# Calcul de toutes les valeurs propres

Deux matrices carrées A et B de même dimension sont dites semblables s&#8217;il existe une matrice $P$ inversible telle que



$$
\mathrm{P}^{-1} \mathrm{AP}=\mathrm{B} .
$$


Deux matrices semblables ont les mêmes valeurs propres. En effet, si $\lambda$ est une valeur propre de $\mathrm{A}$ et $\mathbf{x} \neq \mathbf{0}$ un vecteur propre associé, on a



$$
\mathrm{BP}^{-1} \mathbf{x}=\mathrm{P}^{-1} \mathrm{~A} \mathbf{x}=\lambda \mathrm{P}^{-1} \mathbf{x},
$$


ce qui revient à dire que $\lambda$ est aussi valeur propre de $\mathrm{B}$ et $\mathbf{y}=\mathrm{P}^{-1} \mathbf{x}$ est un vecteur propre associé.


Les méthodes permettant le calcul simultané de toutes les valeurs propres d&#8217;une matrice transforment généralement A (après une infinité d&#8217;itérations) en une matrice semblable diagonale ou triangulaire. Les valeurs propres sont alors simplement les coefficients diagonaux de la matrice obtenue.
## Méthode QR pour le calcul des valeurs propres

Parmi ces méthodes, citons la méthode $Q R$ qui est implémentée dans la fonction `scipy.linalg.qr` et qui est utilisée par la fonction `scipy.linalg.eig`.
La commande `D,V=scipy.linalg.eig(A)` renvoie un vecteur $D$ contenant toutes les valeurs propres de $A$ ainsi que les vecteur propres normalisés à droite $V$.
Le nom de la méthode $QR$ pour le calcul de valeurs propres provient de l&#8217;utilisation répétée de la factorisation $QR$.
Nous ne présentons ici la méthode $QR$ pour les valeurs propres que pour les matrices réelles et sous sa forme la plus simple (dont la convergence n&#8217;est pas toujours garantie).
*Note:* La factorisation $QR$ d&#8217;une matrice $A$ consiste à écrire $A$ comme le produit d&#8217;une matrice orthogonale $Q$ (i.e. $Q^T Q = Q Q ^T = I$) et d&#8217;une matrice triangulaire supérieure $R$.
La factorisation $QR$ est très versatile: c&#8217;est une méthode de décomposition matricielle qui permet de résoudre des systèmes linéaires, de calculer des valeurs propres, de résoudre des problèmes d&#8217;optimisation, etc.
Par exemple la factorisation $QR$ est utilisée pour résoudre des systèmes linéaires $A x = b$ de la manière suivante:

1. On écrit $A$ comme le produit de $Q$ et $R$ en utilisant la factorisation $QR$, puis on cheche $x$  tel que $A x = Q R x = b$.
1. On résout le système $Q c = b$ en calculant $c = Q^T b$.
1. On résout le système $R x = c$ pour $x$.



In [0]:
import numpy as np
from scipy.linalg import qr

alpha = 30
A = np.array([[alpha, 2, 3, 13],
              [5, 11, 10, 8],
              [9, 7, 6, 12],
              [4, 14, 15, 1]])

b=np.array([1,2,3,4])
Q,R=qr(A)
# verification de Q
print(f"Q orthogonale? {np.allclose(Q.T@Q, np.eye(4))}")
c=Q.T@b
x=np.linalg.solve(R,c)
print(x)


x1=np.linalg.solve(A,b)
print(x1)
assert np.allclose(x,x1)


La factorisation $QR$ est également utilisée pour résoudre des problèmes d&#8217;optimisation. Par exemple, la méthode de résolution des moindres carrés  utilise la factorisation $QR$ pour résoudre le problème d&#8217;optimisation. Voici un exemple simple de résolution de moindres carrés:


In [0]:
import numpy as np
from scipy.linalg import qr,solve_triangular

A = np.array([[1, 2], [3, 4], [5, 6]])
b = np.array([1, 2, 3])

Q, R = qr(A)
print(f"Q orthogonale? {np.allclose(Q.T@Q, np.eye(3))}")
print(f"R triangulaire supérieure? {np.allclose(np.triu(R), R)}")
print(f"R={R}")
c = Q.T @ b
print(f"c={c}")
# solve the subsystem Rx = c using only n rows of R
x = solve_triangular(R[:2], c[:2])
print(x)

x1 = np.linalg.lstsq(A, b, rcond=None)[0]
print(x1)
assert np.allclose(x, x1)


Une presentation plus détaillée est disponible dans la [section ci-dessous.](#_décomposition_qr)
Utilisons la méthode $QR$ pour calculer les valeurs propres de la matrice $A$ définie ci-dessous:

1. L&#8217;idée consiste à construire une suite de matrices $\mathrm{A}^{(k)}$, toutes semblables à $\mathrm{A}$.
1. Après avoir posé $\mathrm{A}^{(0)}=\mathrm{A}$, on utilise la factorisation $\mathrm{QR}$ pour calculer les matrices carrées $\mathrm{Q}^{(k+1)}$ et $\mathrm{R}^{(k+1)}$ pour $k=0,1, \ldots$ telles que

$$
\mathrm{Q}^{(k+1)} \mathrm{R}^{(k+1)}=\mathrm{A}^{(k)},
$$
1. puis on pose $\mathrm{A}^{(k+1)}=\mathrm{R}^{(k+1)} \mathrm{Q}^{(k+1)}$.

*Note:* $\mathrm{A}^{(k)}$ est une matrice de Hessenberg, c&#8217;est-à-dire une matrice dont les éléments en dessous de la première sous-diagonale sont nuls.
Elle est semblable à $\mathrm{A}$ et a les mêmes valeurs propres.
Les matrices $\mathrm{A}^{(k)}, k=0,1,2, \ldots$ sont toutes semblables, elles ont donc les mêmes valeurs propres que A.
De plus, si $\mathrm{A} \in \mathbb{R}^{n \times n}$ et si ses valeurs propres vérifient $\left|\lambda_1\right|>\left|\lambda_2\right|>\ldots>\left|\lambda_n\right|$, alors

$$
\lim _{k \rightarrow+\infty} \mathrm{A}^{(k)}=\mathrm{T}=\left[\begin{array}{cccc}
\lambda_1 & t_{12} & \ldots & t_{1 n} \\
0 & \ddots & \ddots & \vdots \\
\vdots & & \lambda_{n-1} & t_{n-1, n} \\
0 & \ldots & 0 & \lambda_n
\end{array}\right]
$$
La vitesse de décroissance vers zéro des coefficients triangulaires inférieurs, $a_{i, j}^{(k)}, i>j$, quand $k$ tend vers l&#8217;infini, dépend de $\max _i\left|\lambda_{i+1} / \lambda_i\right|$.
En pratique, on stoppe les itérations quand $\max _{i>j}\left|a_{i, j}^{(k)}\right| \leq \epsilon$, où $\epsilon>0$ est une tolérance fixée.
*Note:* Si de plus $\mathrm{A}$ est symétrique, la suite $\left\{\mathrm{A}^{(k)}\right\}$ converge vers une matrice diagonale.
## Implémentation basique de la méthode QR

Le Programme ci-dessous implémente la méthode QR. Les paramètres d&#8217;entrée sont la matrice $A$, la tolérance tol et le nombre maximum d&#8217;itérations nmax.


In [0]:
Unresolved directive in 4-qr.adoc - include::example$tan/eig/qrbasic.py[]


## Exemple

*Exemple: famille de matrices $A(\alpha)$*\
Considérons la matrice A(30) et appelons `qrbasic`  pour calculer ses valeurs propres :


In [0]:
from tan.eig.qrbasic import qrbasic
import numpy as np

alpha = 30
A = np.array([[alpha, 2, 3, 13],
              [5, 11, 10, 8],
              [9, 7, 6, 12],
              [4, 14, 15, 1]])
D, iter = qrbasic(A, 1e-14,100)
print(f"La méthode converge en {iter} itérations")
print(D)

from scipy.linalg import eig
D1, _ = eig(A)

#D and D1 should be equal
assert np.allclose(D, D1)


Ces valeurs sont en bon accord avec celles obtenues par la commande `scipy.linalg.eig`.
La vitesse de convergence décroît quand des valeurs propres ont des modules presque identiques. C&#8217;est le cas de la matrice correspondant à $\alpha=-30$ : deux valeurs propres ont à peu près le même module et la méthode a alors besoin de 1149 itérations pour converger avec la même tolérance :


In [0]:
from scipy.linalg import eig
from tan.eig.qrbasic import qrbasic
import numpy as np

alpha = -30
A = np.array([[alpha, 2, 3, 13],
              [5, 11, 10, 8],
              [9, 7, 6, 12],
              [4, 14, 15, 1]])
D, iter = qrbasic(A, 1e-14, 2000)
print(f"La méthode converge en {iter} itérations")
print(D)

D1, _ = eig(A)
indices = np.argsort(np.abs(D1))
# Réorganiser les valeurs propres et vecteurs propres
D1 = D1[indices[::-1]]
print(D1)

# D and D1 should be equal
assert np.allclose(D, D1)


## Décomposition QR

*Théorème 8: (Décomposition QR)*\
Soit $A \in \mathscr{M}_{m,n}({\mathbb{R}})$ avec $m \geqslant n$. Alors il existe $Q \in O_m({\mathbb{C}})$ et $R \in \mathscr{M}_{m,n}({\mathbb{R}})$ triangulaire supérieure telles que $A = QR$.

Si $A$ est inversible (ce qui sous-entend $n=m$), on peut supposer de plus que $R$ est à diagonale strictement positive, et la décomposition est alors unique.

On a un résultat analogue pour ces matrices complexes.
* **Preuve de l’unicité**\
Si $A=Q_1 R_1 = Q_2 R_2$, $A$
étant inversible, $R_1$ et $R_2$ aussi de sorte que
$B=Q_2^T Q_1 = R_2 R_1^{-1}$ est orthogonale et triangulaire
supérieure. Mais alors $B^T B = I_n$ est une décomposition de
Cholevski de $I_n$: par unicité, $B=I_n$ et
$Q_1 = Q_2$, $R_1=R_2$. ◻
On va voir deux méthodes pour construire une décomposition QR.
## Méthode de Householder

*Définition 9*\
Si $u \in {\mathbb{R}}^n \setminus \{0 \}$, on note $H_v$ la matrice de réflexion
par rapport à $v^\perp$:


$$
H_v = I_n - \frac{2}{\| v \|^2} v v^T.
$$

Si $H_v(v) = -v$ et si $w \in v^\perp$,
$H_v(w)=w$.
pour obtenir un endomorphisme hermitien unitaire.
On pose aussi $H_0 = I_n$.

*Note:* Remarque: dans le cas complexe, on pose
$\displaystyle H_v(u) = u - \frac{1}{\|v \|^2} (u^*v + v^*u)$
Soit $A \in \mathscr{M}_n({\mathbb{C}})$.
On note $u_1 = (a_{1,1}, \dots, a_{n,1})^T$ la première
colonne de $A$,
$\alpha = \mathop{\mathrm{\mathrm{sgn}}}(a_{1,1}) \| u_1 \|$
(le signe est pour des raisons de stabilité) et
$v_1 = u_1 - \alpha e_1$ si $u_1$ n’est pas
colinéaire à $e_1$ et $v = 0$ sinon.
Alors on constate que $H_{v_1} u_1 = \alpha e_1$. En effet, on
a $u_1 = v_1 + \alpha e_1$ donc

$$
\alpha^2 = \| u_1 \|^2 = \| v_1 \|^2 + \alpha^2 + 2 \alpha v_1^T e_1, \quad \text{i.e} \quad \| v_1 \|^2 = - 2 \alpha v_1^T e_1,
$$
et il vient
$\displaystyle v_1^T u_1 = \| v_1 \|^2 + \alpha v^T e_1 = \frac{1}{2} \| v_1 \|^2$,
donc

$$
\begin{aligned}
H_{v_1} u_1 = u_1 - 2 \frac{v_1^T u_1}{\| v_1 \|^2} v_1 = u_1 - v_1 = \alpha e_1.
\end{aligned}
$$
Ceci signifie que

$$
\label{eq:house_1}
H_{v_1} A = \begin{pmatrix}
\alpha & * & \cdots & * \\
0 & & \\
\vdots & & A_1 \\
0 &
\end{pmatrix}
$$
On itère alors sur $A_1$, en multipliant à gauche par des
matrices du type latexmath:[H_{v_k} = \begin{pmatrix}
I_{k-1} &amp; 0\\
0  &amp; H_{\tilde v_k} \\
\end{pmatrix}] à la $k$ème itération, où
$\tilde v_k \in {\mathbb{R}}^{m-k+1}$ et $v_k = (0_{k-1}, v_k) \in {\mathbb{R}}^m$.
On constate alors que $H_{v_{m}} \cdots H_{v_1} A$ est
triangulaire supérieure, et
$Q : = (H_{v_{m}} \cdots H_{v_1})^{-1} = H_{v_1}^T \cdots H_{v_m}^T \in O_n({\mathbb{R}})$ (car c’est un groupe).
$Q$ et $R = Q^T A$ conviennent.
Si $A$ est inversible, pour obtenir des coefficients diagonaux
strictement positifs, on reprend la construction avec cette fois
$\alpha = \| u_1 \| >0$ (car $u_1 = Ae_1 \ne 0$).
## Méthode de Givens

On rappelle que pour $\theta \in {\mathbb{R}}$, la matrice
latexmath:[\begin{pmatrix} \cos \theta &amp; - \sin \theta \\
\sin \theta &amp; \cos \theta
\end{pmatrix}] est la matrice de la rotation d’angle $\theta$
du plan.
L’idée est, comme dans la méthode de Householder, d’annuler la première colonne de $A$ grâce à des rotations adaptées. Étant donnés
$\alpha, \beta \in {\mathbb{R}}$ tel que $\alpha^2 + \beta^2 =1$, on considère
les matrices $R_{p,q}(\alpha,\beta))$ dont le coefficient
d’indice $(i,j)$ vaut $\alpha$ si
$i=j \in \{ p,q \}$, $\beta$ si
$(i,j) = (p,q)$, $-\beta$ si
$(i,j) = (q,p)$, $1$ si
$i=j \notin \{ p,q \}$ et $0$ sinon. Il s’agit de la
matrice de rotation d’angle $\theta$ (tel que
$\cos \theta = \alpha$, $\sin \theta = \beta$) dans
le plan $\mathop{\mathrm{\mathrm{Vect}}}(e_p,e_q)$.
On constate que le coefficient $(2,1)$ de
$A_{2,1} = R_{2,1}(\alpha_2, \beta_2) A$ est nul, pour

$$
\alpha_2 = \frac{a_{11}}{\sqrt{a_{1,1}^2 + a_{2,1}^2}}, \quad \beta_2 = \frac{a_{21}}{\sqrt{a_{1,1}^2 + a_{2,1}^2}}
$$
(si $a_{11} = a_{21} =0$, on pose
$(\alpha_2, \beta_2) = (1,0)$. On peut recommencer: on peut
choisir $(\alpha_3,\beta_3)$ de manière similaire, de sorte
que $A_{3,1} = R_{3,1} (\alpha_3, \beta_3) A_{2,1}$ ait un
coefficient $(3,1)$ nul. Vu la manipulation, la nullité du
coefficient $(2,1)$ est préservée.
En itérant sur chaque coefficient de la première colonne, on obtient une
matrice

$$
A_{n,1} = R_{n,1}(\alpha_n, \beta_n) \cdots R_{2,1}(\alpha_2,\beta_2) A
$$
dont la première colonne est nulle (excepté le premier coefficient).
Comme dans le cas de Householder, on peut alors itérer sur chaque
colonne. $SO_n({\mathbb{R}})$ étant un groupe, le produits des rotations
$R_{n,1}(\alpha_n, \beta_n) \cdots R_{2,1}(\alpha_2,\beta_2) \in SO_n({\mathbb{R}})$, et on obtient ainsi une décomposition QR.
## Forme Hessenberg

*Définition 10*\
Une matrice $(a_{ij})_{ij}$ est dite de
Hessenberg si $a_{ij} =0$ dès que $j \leqslant i-2$,
autrement dit, les coefficients non nuls se trouvent sur la 1ère
sous-diagonale et au-dessus.
L’intérêt essentiel est que toute matrice est semblable à une matrice de Hessenberg, via un changement de base *rapide à calculer*.
*Théorème 11*\
Soit $A \in \mathscr{M}_n({\mathbb{C}})$. Il existe $v_1, \dots, v_{n-1} \in {\mathbb{C}}$ tels que


$$
H_{v_{n-1}} \cdots H_{v_1} A H_{v_1}  \cdots H_{v_{n-1}} \text{ soit Hessenberg}.
$$
Pour la première itération, on utilise la méthode de
Householder mais uniquement sur les lignes $2$ à
$n$. Autrement dit, on choisit
$u_1 = (0, a_{2,1} ,\dots, a_{n,1})^T$,
$\alpha = \mathop{\mathrm{\mathrm{sgn}}}(a_{2,1}) \| u_1 \|$
et $v_1 =u_1 - \alpha e_2$, de sorte que


$$
H_{v_1} A =  \begin{pmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
\alpha & \\
0 & & \\
\vdots & & A_1^* \\
0 &
\end{pmatrix}
$$

Alors, comme $v_1$ n’a pas de composante sur $e_1$
(et que la multiplication à droite opère sur les colonnes)
$H_{v_1} A H_{v_1}$ a la même forme (avec une matrice
$A_1$ au lieu de $A_1^*$). Ensuite, on itère sur
$A_1$.
*Note:* La même idée fonctionne avec la méthode de Givens.
*Corollaire 12*\
Si $A$ est symétrique (ou hermitienne), on obtient une matrice tridiagonale symétrique (resp. hermitienne) après la mise sous forme Hessenberg précédente.
En effet, $H_{v}$ est une matrice orthogonale, donc $H_{v} A H_v$ est encore symétrique, et de même en itérant les composées.
## Calcul des valeurs propres

*Algorithme QR:*\
Soit $A \in \mathscr{M}_n({\mathbb{R}})$.

On pose $A_1 = A$. Pour $k \geqslant 1$,


1. On calcule une décomposition QR de $A_k$:
$A_k = Q_k R_k$.
1. On définit $A_{k+1} = R_k Q_k$.

Souvent, on commence par mettre sous forme Hessenberg: on pose $A_1 = P^{-1} A P$, avec $A_1$ Hessenberg.
Le but est de gagner en complexité: le calcul d’une décomposition QR est en $O(n^2)$ pour une forme Hessenberg contre $O(n^3)$ en général pour une matrice pleine.
*Lemme 13*\
$A_k$ et $A$ sont semblables.
En particulier, $\mathop{\mathrm{\mathrm{Sp}}}(A_k) = \mathop{\mathrm{\mathrm{Sp}}}(A)$.

Si $A_1$ est sous forme Hesseberg, $A_k$ aussi pour $k \geqslant 1$.
En effet, $R_k = Q_k^{-1} A_k = Q_k^T A_k$ donc $A_{k+1} = R_k Q_k = Q_k^{T} A_k Q_k$.
Ainsi $A_{k+1}$ est (orthogonalement) semblable à $A_k$.

Si $A_k$ est sous forme Hessenberg,
$Q_k = A_k R_k^{-1}$ également (car $R_k$ est
triangulaire), donc $A_{k+1} = R_kQ_k$ aussi.

On conclut par récurrence.
*Théorème 14*\
On suppose que $A \in \mathscr{M}_n({\mathbb{C}})$ vérifie: il existe $P \in GL_n({\mathbb{C}})$ qui admet une décomposition LU et


$$
P^{-1} A P = D : = \mathop{\mathrm{\mathrm{diag}}}(\lambda_1, \dots, \lambda_n),
$$

avec $|\lambda_1| > |\lambda_2| > \cdots > |\lambda_n| >0$.
(En particulier, $A$ est inversible et diagonalisable à
valeurs propres simples).

Alors, quand $k \to +\infty$,
$(A_k)_{i,i} \to \lambda_i$ et $(A_k)_{i,j} \to 0$
pour tous $j <i$.
Remarquons que $Q_{k+1} R_{k+1} = A_{k+1} = R_k Q_k$.

Notons $O_k = Q_1 \cdots Q_k$ et
$S_k = R_k \cdots R_1$. Alors $O_k$ est orthogonale
et $R_k$ est triangulaire supérieure,
$A_k = O_{k-1}^T A O_{k-1}$ et de plus, on a la décomposition
QR


$$
A^k = O_k S_k.
$$

En effet, on a par exemple


$$
\begin{gathered}
O_2 S_2 = Q_1 Q_2 R_2 R_1 = Q_1 (R_1 Q_1) R_1 = (Q_1 R_1) (Q_1 R_1) = A^2 \\
O_3 S_3 = Q_1 Q_2 (Q_3 R_3) R_2 R_1 =  Q_1 Q_2 (R_2 Q_2) R_2 R_1 =  Q_1 (R_1 Q_1) (R_1Q_1)  R_1 = A^3
\end{gathered}
$$

et ainsi de suite.

D’autre part, soit $P^{-1} = LU$ la factorisation
$LU$ de $P^{-1}$ (qui se déduit de celle de
$P$) et $P=QR$ avec $R_{ii}>0$ (possible
car $P$ est inversible). Alors on a


$$
\begin{aligned}
A^k = P D^k P^{-1} = QR D^k LU = Q (R D^k L D^{-k}   R^{-1}) R D^k U,
\end{aligned}
$$

et on calcule que $D^k L D^{-k}$ est triangulaire inférieure,
et


$$
(D^k L D^{-k})_{ij} = \left( \frac{\lambda_i}{\lambda_j} \right)^k L_{ij} \quad \text{si } i \geqslant j.
$$

Par hypothèse sur les $\lambda_i$, et comme
$L_{ii} =1$, cela entraine
$RD^k L D^{-k} R^{-1}= I_n + N_k$ avec $N_k \to 0$.

Soit $I_n + N_k = \tilde Q_k \tilde R_k$ une factorisation
$QR$ avec $(\tilde R_{k})_{ii}>0$ (possible car
$I_n +N_k$ est inversible pour $k$ assez grand).

$O_n({\mathbb{R}})$ étant compact, on peut extraire de $\tilde Q_k$ une sous
suite convergente: $\tilde Q_{\phi(k)} \to \tilde Q \in O_n({\mathbb{R}})$. Par conséquent, $\tilde R_{\phi(k)} \to \tilde Q^T$ est
triangulaire supérieure et orthogonale: elle est donc diagonale, à
coefficients $\pm 1$. Mais on a aussi
$\tilde R_{\phi(k), ii} >0$ donc
$\tilde R_{ii} \geqslant 0$ et donc
$\tilde R = I_n = \tilde Q$.

La limite ne dépendant pas de la sous-suite considérée, on a convergence
de toute la suite: $\tilde Q_k, \tilde R_k \to I_n$.

Alors


$$
A^k = Q (\tilde Q_k \tilde R_k) R D^k U = (Q \tilde Q_k)  (\tilde R_k R D^k U).
$$

C’est une autre décomposition QR de $A^k$. Par unicité de la
décomposition, il existe une matrice
$D_k = \mathop{\mathrm{\mathrm{diag}}}(\varepsilon_{i,k})$ où
$\varepsilon_{i,k} \in \{ \pm 1\}$ adéquate, telle que


$$
O_k = Q \tilde Q_k D_k.
$$

Faisons le lien avec $A_k$: comme
$A = PDP^{-1} = QR DR^{-1}Q^{-1}$,


$$
\begin{aligned}
A_{k+1} & = O_{k}^T A O_{k} = ( Q \tilde Q_k D_k)^T (QRDR^{-1}Q^{-1})  Q \tilde Q_k D_k \\
& = D_k \tilde Q_k RDR^{-1} \tilde Q_k D_k
\end{aligned}
$$

Mais $\tilde Q_k RDR^{-1} \tilde Q_k \to RDR^{-1}$ qui est
triangulaire avec la même diagonale que $D$:
$\lambda_1, \dots, \lambda_n$. On conclut que
$(A_{k})_{ii} \to \lambda_i$ et $(A_k)_{ij} \to 0$
pour $i \geqslant j$.
*Note:* On constate que la convergence est en $O(\lambda^k)$ où $\lambda = \min |\lambda_{k+1}/\lambda_{k}|$.
On peut améliorer la convergence avec quelques astuce comme des shifts (en considérant $A-\mu I_n$) ou des déflations (si $A$ est triangulaire par blocs).
## Cas des matrices symétriques

Dans le cas où $A$ est une matrice symétrique, la forme de Hessenberg $B$ de A donnée par le [théorème ci-dessus](#thm:hessenberg) est tridiagonale (et symétrique).
Notons pour $k =1, \dots, n$

$$
B_k = \begin{pmatrix}
b_1 & c_1 & 0  \\
c_1 & b_2 & c_2 & \\
0 &   c_2 & \ddots & \ddots \\
 & &   \ddots & \ddots & c_{k-2} & 0 \\
& & & c_{k-2} & b_{k-1} & c_{k-1} \\
& &  & 0 & c_{k-1} & b_k
\end{pmatrix},
$$
de sorte que $B_n = B$.
On peut supposer que les $c_i$ sont tous non nuls (sinon on se ramène à un bloc).
On note $p_k = \chi_{B_k}$ le polynôme caractéristique de $B_k$, et $p_0 = 1$.
*Proposition 16*\
On a $p_1(X) = b_1-X$ et pour $k = 2, \dots, n$,


$$
p_k(X) = (b_k - X)p_{k-1}(X) - c_{k-1}^2 p_{k-2}(X).
$$

$p_k$ est scindé à racines simple sur
${\mathbb{R}}$, et ses racines sont séparées par celle de
$p_{k-1}$.
La relation de récurrence est obtenue en développant le déterminant par rapport à la dernière ligne.

On voit que $p_k$ a pour terme dominant
$(-1)^k X^k$, de sorte que $p_k \to +\infty$ en
$-\infty$.

Pour les racines, on raisonne par récurrence forte sur $k$. Si
$\lambda$ est une racine de $p_{k-1}$, alors
$p_{k}(\lambda) = - c_{k-1}^2 p_{k-2}(\lambda)$.
$p_{k}(\lambda)$ et $p_{k-1}(\lambda)$ sont des
signes opposés, non nuls.

Si $\lambda_{0,k-1}$ est la plus petite racine de
$p_{k-1}$, alors $p_{k-2} >0$ sur
$]-\infty, \lambda_{0,k-1}$] par hypothèse de récurrence, donc
$p_{k}(\lambda_{0,k-1}) <0$ et $p_k$ admet une
racine $\lambda_{0,k} < \lambda_{0,k-1}$. Puis on construit de
proche en proche les racines de $p_k$ en vérifiant
l’entrelacement.
On définit $s(k,\mu)$: c’est le signe de $p_k(\mu)$ si ce nombre est non nul, et celui de $p_{k-1}(\mu)$ sinon; et $\mathscr{N}(k,\mu)$ est le nombre de changements de signe dans

$$
(+, s(1,\mu), s(2,\mu), \dots, s(k,\mu)).
$$
*Corollaire 17*\
$\mathscr{N}(k,\mu)$ est le nombre de racines de $p_k$ inférieures strictement à $\mu$.
Soit $k \in \left[\!\!\left[ 1, n \right]\!\!\right]$. On déduit de ce qui précède le calcul de la $k$ème valeur propre $\lambda_k$ de $A$ par dichotomie.
*Algorithme:*\
Soit $\varepsilon>0$ et $[a_0,b_0$] un intervalle
contenant $\lambda_k$.

Tant que $|b_i-a_i| > \varepsilon$

$c_i = \displaystyle\frac{a_i+b_i}{2}$,

Si $\mathscr{N}(n,c_i) \geqslant k$ alors
$\lambda_k \in [a_i,c_i]$, on pose $a_{i+1} = a_i$,
$b_{i+1} = c_{i}$.

Sinon $\lambda_k \in [c_i,b_i]$, on pose $a_{i+1} = c_i$, $b_{i+1} = b_{i}$.
